In [ ]:
import os
import pandas as pd
import pickle

%config Completer.use_jedi = False

joinpath = os.path.join

In [ ]:
rawdatainfo = pd.read_csv('datamanager/overview_rawdata.csv', sep=';')
rawdatainfo

In [ ]:
file = '../DataVisDreamDest/data/computed_data/unloc.pickle'
with open(file, 'rb') as f:
    unloc = pickle.load(f)
unloc.ccode.isna().sum()

In [ ]:
raw_data_path = 'data/01_raw_data/'

!ls {raw_data_path}

In [ ]:
!head {os.path.join(raw_data_path,'Index.csv ') }

In [ ]:
for file in os.listdir(raw_data_path): 
    print('\n', file)
    !head {os.path.join(raw_data_path, file) }

In [ ]:
rawdatainfo = pd.read_csv(joinpath('datamanager/overview_rawdata.csv'))

In [ ]:
citytemp = pd.read_csv

In [ ]:
from datamanager import processing
cleaner = getattr(processing, f'clean_{citytemp}')

cleaner()

In [ ]:
rawdatainfo

In [ ]:
numdata = pd.DataFrame(columns=['location_name', 'location_type'])


for idx, row in rawdatainfo.iterrows():
    print(f'Processing file {filename}')

    
    #
    cleaner = getattr(processing, f'clean_{row.dataset}')
    
    # read file
    file_raw = pd.read_csv()
    
    # apply cleaning
    file = cleaner(file_raw)
    
    # join 
    numdata.merge(how='outer')
    print(idx, row)
    #print(datafile.name[idx])